In [1]:
from utils import transform_player_data

import json

In [2]:
player_data_dict = transform_player_data.transform()

json.dump(
    player_data_dict, 
    open('data/player_data_dict.json', "w"), 
    indent = 4)